# 划分数据

In [ ]:
import os
import re
import shutil
import pandas as pd
from onekey_algo.custom.components.comp2 import split_dataset4sol
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.utils import print_join_info

clinic = pd.read_csv(r'E:/20240601-BiQiu/label.csv')
data = clinic[['ID', 'label', 'group']]
train_data = data[data['group'] == 'train']
test_data = data[data['group'] != 'train']

rt = split_dataset4sol(train_data, train_data['label'], cv=False, n_trails=10, test_size=0.3, save_dir='.', shuffle=True)
for idx, (train, val) in enumerate(rt):
    val['group'] = 'val'
    rnd = pd.concat([train, val, test_data], axis=0)
    display(rnd['group'].value_counts())
    rnd.to_csv(f'split_info/label-RND-{idx}.csv', index=False)
#     pd.merge(rnd[rnd['group'] == 'train'], samples.drop_duplicates('ID'), on='ID', how='inner')[['ID', 'label']].to_csv(f'split_info/train3d-RND-{idx}.txt', 
#                                                                                                   sep='\t', header=None, index=None)
#     pd.merge(rnd[rnd['group'] != 'train'], samples.drop_duplicates('ID'), on='ID', how='inner')[['ID', 'label']].to_csv(f'split_info/val3d-RND-{idx}.txt',
#                                                                                                   sep='\t', header=None, index=None)

# 拆分病理

In [ ]:
import pandas as pd
import os
import numpy as np
from glob import glob
from onekey_algo.custom.components.comp2 import split_dataset4sol
from onekey_algo import get_param_in_cwd

root = r'E:/20240601-BiQiu/Pathomics'
patches = pd.DataFrame(glob(os.path.join(root, 'patches_norm', '*', '*.jpg')), columns=['fpath'])
patches['ID'] = patches['fpath'].map(lambda x: os.path.basename(os.path.dirname(x)))
patches['filename'] = patches['fpath'].map(lambda x: os.path.basename(x))
patches                            

In [ ]:
import pandas as pd

nact = pd.read_csv(r'E:/20240601-BiQiu/NACT.csv')
nact = nact[nact['NACT'] == 0]
for idx in range(10):
    l = pd.merge(pd.read_csv(f'split_info/label-RND-{idx}.csv'), nact[['ID']], on='ID', how='inner')
    l['ID'] = l['ID'].map(lambda x: x.replace('.nii.gz', ''))
    patch_info = pd.merge(patches, l, on='ID', how='inner')
    os.makedirs(os.path.join(root, 'split_info'), exist_ok=True)
    train = patch_info[patch_info['group'] == 'train'][['fpath', 'ID', 'label']]
    train[['fpath', 'label']].to_csv(os.path.join(root, 'split_info', f'train-RND-{idx}.txt'),  index=False, header=False, sep='\t')
    valtest = patch_info[patch_info['group'] != 'train'][['fpath', 'ID', 'label']]
    valtest[['fpath', 'label']].to_csv(os.path.join(root, 'split_info', f'val-RND-{idx}.txt'), index=False, header=False, sep='\t')    
    print(f"随机划分：{idx}，训练集：{train.shape[0]}, {len(np.unique(train['ID']))}, 测试集集：{valtest.shape[0]}, {len(np.unique(valtest['ID']))}")

In [ ]:
import pandas as pd

set(pd.read_csv(r'E:/20240601-BiQiu/label.csv')['ID']) - set(pd.read_csv(r'group.csv')['ID'])

In [ ]:
import os

log = pd.concat([pd.read_csv(f'results/Path_LR_{s}.csv') for s in ['train', 'val', 'test']], axis=0)
display(log)
set(pd.read_csv(r'features/path1_pred_histogram.csv')['ID']) - set(log['ID'])

In [ ]:
import os

log = pd.concat([pd.read_csv(f'results/Habitat_LR_{s}.csv') for s in ['train', 'val', 'test']], axis=0)
display(log)
aid = pd.merge(pd.read_csv('E:/20240601-BiQiu/label.csv'), pd.DataFrame(os.listdir(r'E:/20240601-BiQiu/Radiomics/ADC/'), columns=['ID']))
set(aid['ID']) - set(log['ID'])

In [ ]:
nact = pd.read_csv(r'E:/20240601-BiQiu/NACT.csv')
nact = nact[nact['NACT'] == 0]

set(nact['ID']) - set(pd.read_csv('data/clinic_sel.csv')['ID'])